# Convolutional Encoder/Decoder.

In [3]:
%matplotlib inline
from pylab import *
import pamfun as pamfun
import ascfun as af

__Below is the code written to generate the convolution code for a bit sequence with binary rate 1/n__

In [25]:
def ccencod10(di, GD, trim=True):
    """
    Binary rate 1/n convolutional encoder with transfer
    function matrix G(D), V 1.0
    >>>>> ci = ccencod10(di, GD, trim) <<<<<
    where ci: multiplexed binary code sequence
    di: binary (unipolar) data sequence
    GD: array of n encoder polynomials
    trim: if True, trim ci to n*len(di)
    Examples:
    GD = [[1,0],[1,1]] for G(D) = [1 1+D]
    GD = [[1,0,1],[1,1,1]] for G(D) = [1+D^2 1+D+D^2]
    GD = [[1,0,1,1],[1,1,0,1],[1,1,1,1]]
    for G(D) = [1+D^2+D^3 1+D+D^3 1+D+D^2+D^3]
    """
    code = array([])                                         # initialisation of code
    nm = len(GD[0])-1                                        # number of memory elements
    ps = zeros(nm)                                           # present state
    ns = zeros(nm)                                           # next state
    
    # Generating the convolution code
    for bit in di:
        GD_prime=append(bit,ps)*GD
        code = append(code,sum(GD_prime,axis=1)%2)
        ns = append(bit,ps)[0:nm]                            # Calculating the next state
        ps = ns     
    code = array(code,int8)
    if trim==True:
        code = code[0:(len(GD)*len(di))+1]
    return code
    

__Now checking the code functionality by comparing the codes encoding given in the lab manual.__

In [5]:
di = array([1, 0, 1, 1, 0, 0])                             # Example Code given in the Experiment 2
GD = array([[1,0],[1,1]])                                  # the transfer function matrix

In [6]:
ccencod10(di, GD, trim=True)

array([1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0], dtype=int8)

__Now verifying the code using the di and GD given at Page 11 & 12 of lab manual.__

In [7]:
di = array([1, 1, 1, 0, 0, 0]) 
GD = array([[1,0,1],[1,1,1]])

In [8]:
ccencod10(di, GD, trim=True)

array([1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0], dtype=int8)

__We found the outputs of the ccencod10 to the enocded ouput of the di from lab manual are same.__ 

__Below is the given code (in lab manual) for decoding the convolution code__

In [41]:
def ccdecod10(bi):
    """
    Viterbi decoder for binary rate 1/2 convolutional code received
    as polar (binary 0 -> -1, binary 1 -> +1) PAM signal from AWGN
    channel.
    Version 1.0 for encoder transfer function matrix G(D) = [1 1+D]
    >>>>> dihat, DM = ccdecod10(bi) <<<<<
    where dihat: ML estimate of binary data sequence
    DM: array of final distance metrics
    bi: received noisy polar binary (+A/-A) sequence
    """
    n, K, m = 2, 2, 1                                                  # Rate 1/n, constraint len K, memory m
    N = int(floor(len(bi)/float(n)))                                   # Number of codeword frames
    CBM = [[-1,-1],[+1,+1],[-1,+1],[+1,-1]]                            # Code-bit (-1/+1) matrix
    DM = array([0,1000])                                               # (Initial) distance metrics
    dA = array(zeros((2,1)),int)                                       # Competing data sequence array
    ix2 = array([0,0,1,1],int)                                         # State indexes (doubled)
    for i in range(N):
        bDM = np.power(outer(ones(4),bi[n*i:n*(i+1)]) - CBM,2.0)
        bDM = dot(bDM,ones((n,1)))                                     # Branch distance metrics
        bDM = reshape(bDM,size(bDM))                                   # Convert to 1d array
        tDM = DM[ix2] + bDM                                            # Tentative distance metrics
    
    tDM = reshape(tDM,(2,2))                                           # Reshape for path elimination
    DM = amin(tDM,axis=0)                                              # Select paths with smaller metric
    ix = argmin(tDM,axis=0)                                            # Indexes of smaller metric paths
    dA = hstack((dA[ix,:],array([[0],[1]])))                           # Competing data sequence update
    dA = dA[:,1:]                                                      # Discard first (dummy) column
    ix = argmin(DM)                                                    # Index of smallest metric
    dihat = dA[ix,:]                                                   # ML-decoded data sequence
    dihat = reshape(array(dihat),size(dihat))                          # Convert to 1d array
    
    return dihat, DM

__Verifying the decoder: We will generate a test sequence, convert it to waveform using rect pulse and then add noise to it. We will try to get back original sequence using the viterbi algorithm__

In [42]:
dn = af.asc2bin('MyTest')
an = ccencod10(dn,array([[1,0,1],[1,1,1]]))
tt, st = pamfun.pam12(an, FB=10000, Fs=10000, ptype='rect', pparms=[])


In [43]:
rt = st + randn(len(st))

In [44]:
dihat,DM=ccdecod10(rt)

In [45]:
dihat

array([1])